In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

In [6]:
epl_1920 = pd.read_csv('E0.csv')
epl_1920 = epl_1920[['HomeTeam','AwayTeam','FTHG','FTAG']]
epl_1920 = epl_1920.rename(columns={'FTHG':'HomeGoals','FTAG':'AwayGoals'})
epl_1920.head()

,HomeTeam,AwayTeam,HomeGoals,AwayGoals
0,Liverpool,Norwich,4,1
1,West Ham,Man City,0,5
2,Bournemouth,Sheffield United,1,1
3,Burnley,Southampton,3,0
4,Crystal Palace,Everton,0,0


In [7]:
epl_1920 = epl_1920[:-10]
epl_1920.mean()

HomeGoals    1.508108
AwayGoals    1.197297
dtype: float64

In [8]:
skellam.pmf(0.0, epl_1920.mean()[0],epl_1920.mean()[1])

0.25428088854445496

In [9]:
skellam.pmf(1, epl_1920.mean()[0],epl_1920.mean()[1])

0.22372371409045236

In [10]:
epl_1920['HomeTeam']

0           Liverpool
1            West Ham
2         Bournemouth
3             Burnley
4      Crystal Palace
            ...      
365            Wolves
366           Watford
367       Aston Villa
368        Man United
369         Liverpool
Name: HomeTeam, Length: 370, dtype: object

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
goal_model_data = pd.concat([epl_1920[['HomeTeam','AwayTeam','HomeGoals']].assign(home=1).rename(
            columns={'HomeTeam':'team', 'AwayTeam':'opponent','HomeGoals':'goals'}),
           epl_1920[['AwayTeam','HomeTeam','AwayGoals']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayGoals':'goals'})])

poisson_model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                  740
Model:                            GLM   Df Residuals:                      700
Model Family:                 Poisson   Df Model:                           39
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1028.6
Date:                Tue, 08 Sep 2020   Deviance:                       748.82
Time:                        08:20:12   Pearson chi2:                     642.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.1498      0.207      0.722      0.470      -0.257       0.556
team[T.Aston Villa]             -0.2673      0.210     -1.274      0.203      -0.679       0.144
team[T.Bournemouth]             -0.3520      0.215     -1.640      0.101      -0.773       0.069
team[T.Brighton]                -0.3663      0.215     -1.707      0.088      -0.787       0.054
team[T.Burnley]                 -0.2359      0.207     -1.140      0.254      -0.642       0.170
team[T.Chelsea]                  0.2312      0.184      1.255      0.210      -0.130       0.592
team[T.Crystal Palace]          -0.5752      0.229     -2.514      0.012      -1.024      -0.127
team[T.Everton]                 -0.2003      0.206     -0.974      0.330      -0.603       0.203
team[T.Leicester]                0.2141      0.184      1.162      0.245      -0.147       0.575
team[T.Liverpool]                0.4126      0.177      2.336      0.019       0.066       0.759
team[T.Man City]                 0.6004      0.171      3.506      0.000       0.265       0.936
team[T.Man United]               0.1620      0.186      0.871      0.384      -0.203       0.527
team[T.Newcastle]               -0.3660      0.215     -1.706      0.088      -0.787       0.055
team[T.Norwich]                 -0.7044      0.240     -2.938      0.003      -1.174      -0.234
team[T.Sheffield United]        -0.3521      0.213     -1.654      0.098      -0.769       0.065
team[T.Southampton]             -0.1003      0.200     -0.502      0.615      -0.492       0.291
team[T.Tottenham]                0.1110      0.189      0.588      0.557      -0.259       0.481
team[T.Watford]                 -0.4427      0.220     -2.014      0.044      -0.874      -0.012
team[T.West Ham]                -0.0796      0.200     -0.399      0.690      -0.471       0.312
team[T.Wolves]                  -0.0563      0.197     -0.287      0.774      -0.441       0.329
opponent[T.Aston Villa]          0.3609      0.192      1.875      0.061      -0.016       0.738
opponent[T.Bournemouth]          0.3232      0.194      1.669      0.095      -0.056       0.703
opponent[T.Brighton]             0.1332      0.202      0.660      0.509      -0.262       0.529
opponent[T.Burnley]              0.0319      0.207      0.154      0.877      -0.373       0.437
opponent[T.Chelsea]              0.1831      0.201      0.911      0.362      -0.211       0.577
opponent[T.Crystal Palace]       0.0502      0.206      0.244      0.807      -0.353       0.453
opponent[T.Everton]              0.1328      0.202      0.658      0.510      -0.263       0.528
opponent[T.Leicester]           -0.1376      0.218     -0.631      0.528      -0.565       0.290
opponent[T.Liverpool]           -0.3271      0.231     -1.419      0.156    

In [13]:
poisson_model.predict(pd.DataFrame(data={'team': 'Liverpool', 'opponent': 'Norwich',
                                       'home':1},index=[1]))

1    3.423612
dtype: float64

In [14]:
poisson_model.predict(pd.DataFrame(data={'team': 'Norwich', 'opponent': 'Liverpool',
                                       'home':0},index=[1]))

1    0.414053
dtype: float64

In [15]:
def simulate_match(foot_model, homeTeam, awayTeam, max_goals=10):
    home_goals_avg = foot_model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = foot_model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
simulate_match(poisson_model, 'Liverpool', 'Norwich', max_goals=3)

array([[0.02154386, 0.00892029, 0.00184673, 0.00025488],
       [0.07375783, 0.03053961, 0.0063225 , 0.00087262],
       [0.12625908, 0.05227789, 0.0108229 , 0.00149375],
       [0.14408735, 0.05965973, 0.01235113, 0.00170467]])

In [16]:
liv_nor = simulate_match(poisson_model, "Liverpool", "Norwich", max_goals=10)
# Liverpool win
np.sum(np.tril(liv_nor, -1))

0.9141605417246046

In [17]:
# draw
np.sum(np.diag(liv_nor))

0.06477098410083065

In [18]:
# Norwich win
np.sum(np.triu(liv_nor, 1))

0.020212436081052984